This notebook has been used to create the five models that compose the final ensemble of 5 models submitted on Codalab.
The peculiarities are:
- They are all based on ConvNextBase pretrained model
- To some of them we applied resizing of the input images
- To some of them we applied augmentation techniques during the training phase

Imports

In [ ]:
seed = 75

import shutil
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.models import Sequential

tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
dataset = np.load('clean_collection.npz', allow_pickle=True)
data = dataset['data']
labels = dataset['labels']

# Cast string labels into integer values
casted_labels = []
for label in labels:
  if label == "unhealthy":
    casted_labels.append(1)
  elif label == "healthy":
    casted_labels.append(0)
  else:
    raise Exception("Invalid label")

# Expand the labels dimension moving from (x,) to (x, 1), with x cardinality
casted_labels = np.expand_dims(casted_labels, axis=-1)

print("Training-Validation-Test Data Shape:", data.shape)
print("Training-Validation-Test Casted Labels Shape:", casted_labels.shape)

# Inspect the target
print('Counting occurrences of target classes:')
unique, counts = np.unique(casted_labels, return_counts=True)
print(dict(zip(unique, counts)))

x_train, x_val, y_train, y_val = train_test_split(data, casted_labels, random_state=seed, test_size=1000, stratify = casted_labels)

del casted_labels
del dataset
del data
del labels
del unique
del counts

In [ ]:
# Divide the heatlhy images from the unhealthy images in the training set
# to see the total number of healthy images and unhealthy images in the training set

x_unhealthy = []
y_unhealthy = []
x_healthy = []
y_healthy = []

for ind in range(len(x_train)):
  if y_train[ind][0] == 1:
    x_unhealthy.append(x_train[ind])
    y_unhealthy.append(1)
  elif y_train[ind][0] == 0:
    x_healthy.append(x_train[ind])
    y_healthy.append(0)
  else:
    raise Exception("Impossible")

print(f"Len train set: {len(x_train)}  num unhealthy: {len(x_unhealthy)} num healthy: {len(x_healthy)}")

In [ ]:
# Balance the training set by adding duplicates of unhealthy images
# To avoid to duplicate the same unhealthy image more than one time
# i store the indexes of the images duplicated in a list

indexes = []                                                    # stores the indexes of the unhealthy images already duplicated one time

while True:
  # When the number of duplicated images is equal to the difference between healthy and unhealthy images
  # it means that we have completely balanced the training dataset
  if len(indexes) >= len(x_healthy) - len(x_unhealthy):
    break

  tmp = np.random.randint(0, len(x_unhealthy))
  if tmp not in indexes:
    indexes.append(tmp)

# We add the duplicated unhealthy images to the training set
for ind in indexes:
  x_unhealthy.append(x_unhealthy[ind])
  y_unhealthy.append(1)

# Combining the healthy and unhealthy images of the training set into one single set
x_train = np.concatenate((x_unhealthy, x_healthy), axis=0)
y_train = np.concatenate((y_unhealthy, y_healthy), axis=0)

y_train = np.expand_dims(y_train, axis=-1)


print(f"Number of unhealthy images in the training set: {len(x_unhealthy)}")
print(f"Number of healthy images in the training set: {len(x_healthy)}")

del indexes
del x_unhealthy, x_healthy, y_unhealthy, y_healthy

Transfer Learing

In [ ]:
# Image augmentation layer composed of multiple techniques
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.17),
        layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
        layers.RandomFlip(),
    ],
    name="img_augmentation",
)

In [ ]:
def build_model(pretrained_model, resize, augmentation_enabled, denses, units, dropout_rate):
  '''
  Build and return the model.
  - pretrained_model is the pretrained model taken from keras.applications
  - resize is the new size of input images
  - augmentation_enabled is a boolean specifying wether the augmentation layer has to be applied
  - denses is the number of dense layers in the top part of the network excluding the last dense layer composed of the single output neuron
  - units is the number of units of the first dense layer (the number of units of the following dense layers is halved each time)
  - dropout_rate is the dropout rate
  '''

  # the model's expected input size
  input_shape = (96, 96, 3)

  # The input layer
  inputs = layers.Input(shape=input_shape)

  if resize != 96:
    # Resizing layer used to resize input images
    resizing = layers.Resizing(height = resize, width = resize, interpolation='bilinear', crop_to_aspect_ratio = True)
    # Perform resize to the wanted dimension
    x = resizing(inputs)

    if augmentation_enabled:
      # Perform augmentation (only during training!)
      x = img_augmentation(x)

    # ConvNextBase model
    x = pretrained_model(x)

  elif augmentation_enabled:
    # Perform augmentation (only during training!)
    x = img_augmentation(inputs)
    # ConvNextBase model
    x = pretrained_model(x)

  else:
    # ConvNextBase model
    x = pretrained_model(inputs)

  # Let's rebuild the top

  x = layers.GlobalAveragePooling2D()(x)

  if(denses != 0):

    # For each dense layer we specify the number of neurons, the kernel_initializer function, a batch normalization layer, the ReLU activation function and the dropout
    for ind in range(0, denses):
      x = layers.Dense(units=int(units/(2**ind)), kernel_initializer=keras.initializers.HeUniform(seed=seed))(x)
      x = layers.BatchNormalization()(x)
      x = layers.ReLU()(x)
      x = layers.Dropout(dropout_rate, seed=seed)(x)

  # The output layer
  outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

  return tf.keras.Model(inputs, outputs, name="model")

In [ ]:
# We can define multiple configurations
# We would train and save a new model for each specified configuration
configs = [
    # ConvNext is based on the ConvNextBase pretrained model and we didn't apply both augmentation and resizing
    {
      'config_name' : "ConvNext",                                         # Configuration name (the name of the model directory after the save)
      'resize' : 96,                                                      # Resize value
      'augmentation_enabled': False,                                      # Wether augmentation has to be used or not during training
      'batch_size' : 64,                                                  # Batch size
      'denses' : 2,                                                       # Number of dense layers
      'units' : 64,                                                       # Number of units in the first dense layer
      'dropout_rate' : 1/8,                                               # Dropout rate
      'opt_transf_learn' : keras.optimizers.Adam(learning_rate = 0.01),   # Optimizer to use in the transfer learning phase
      'opt_fine_tuning' : keras.optimizers.Adam(),                        # Optimizer to use in the fine tuning phase
      'num_layer_to_freeze' : 180                                         # Number of layer to freeze during the fine tuning phase
    },
    # ConvNext_AUG is based on the ConvNextBase pretrained model and we didn't apply resizing, but we applied augmentation
    {
      'config_name' : "ConvNext_AUG",                                     # Configuration name (the name of the model directory after the save)
      'resize' : 96,                                                      # Resize value
      'augmentation_enabled': True,                                       # Wether augmentation has to be used or not during training
      'batch_size' : 64,                                                  # Batch size
      'denses' : 2,                                                       # Number of dense layers
      'units' : 64,                                                       # Number of units in the first dense layer
      'dropout_rate' : 1/8,                                               # Dropout rate
      'opt_transf_learn' : keras.optimizers.Adam(learning_rate = 0.01),   # Optimizer to use in the transfer learning phase
      'opt_fine_tuning' : keras.optimizers.Adam(),                        # Optimizer to use in the fine tuning phase
      'num_layer_to_freeze' : 180                                         # Number of layer to freeze during the fine tuning phase
    },
    # ConvNext_AUG_128 is based on the ConvNextBase pretrained model and we applied resizing to 128x128 and augmentation
    {
      'config_name' : "ConvNext_AUG_128",                                 # Configuration name (the name of the model directory after the save)
      'resize' : 128,                                                     # Resize value
      'augmentation_enabled': True,                                       # Wether augmentation has to be used or not during training
      'batch_size' : 64,                                                  # Batch size
      'denses' : 2,                                                       # Number of dense layers
      'units' : 64,                                                       # Number of units in the first dense layer
      'dropout_rate' : 1/8,                                               # Dropout rate
      'opt_transf_learn' : keras.optimizers.Adam(learning_rate = 0.01),   # Optimizer to use in the transfer learning phase
      'opt_fine_tuning' : keras.optimizers.Adam(),                        # Optimizer to use in the fine tuning phase
      'num_layer_to_freeze' : 180                                         # Number of layer to freeze during the fine tuning phase
    },
    # ConvNext_AUG_200 is based on the ConvNextBase pretrained model and we applied resizing to 200x200 and augmentation
    {
      'config_name' : "ConvNext_AUG_200",                                 # Configuration name (the name of the model directory after the save)
      'resize' : 200,                                                     # Resize value
      'augmentation_enabled': True,                                       # Wether augmentation has to be used or not during training
      'batch_size' : 64,                                                  # Batch size
      'denses' : 2,                                                       # Number of dense layers
      'units' : 64,                                                       # Number of units in the first dense layer
      'dropout_rate' : 1/8,                                               # Dropout rate
      'opt_transf_learn' : keras.optimizers.Adam(learning_rate = 0.01),   # Optimizer to use in the transfer learning phase
      'opt_fine_tuning' : keras.optimizers.Adam(),                        # Optimizer to use in the fine tuning phase
      'num_layer_to_freeze' : 180                                         # Number of layer to freeze during the fine tuning phase
    },
    # ConvNext_AUG_250 is based on the ConvNextBase pretrained model and we applied resizing to 250x250 and augmentation
    {
      'config_name' : "ConvNext_AUG_250",                                 # Configuration name (the name of the model directory after the save)
      'resize' : 250,                                                     # Resize value
      'augmentation_enabled': True,                                       # Wether augmentation has to be used or not during training
      'batch_size' : 64,                                                  # Batch size
      'denses' : 2,                                                       # Number of dense layers
      'units' : 64,                                                       # Number of units in the first dense layer
      'dropout_rate' : 1/8,                                               # Dropout rate
      'opt_transf_learn' : keras.optimizers.Adam(learning_rate = 0.01),   # Optimizer to use in the transfer learning phase
      'opt_fine_tuning' : keras.optimizers.Adam(),                        # Optimizer to use in the fine tuning phase
      'num_layer_to_freeze' : 180                                         # Number of layer to freeze during the fine tuning phase
    }
]

In [ ]:
# Standard parameters

patience = 15
epochs_transf_learn = 10
epochs_fine_tuning = 200

lr_scheduler = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',                                             # Metric to monitor (validation accuracy in this case)
    patience=3,                                                         # Number of epochs with no improvement after which learning rate will be reduced
    factor=0.9,                                                         # Factor by which the learning rate will be reduced (0.9 in this case)
    mode='max',                                                         # Mode to decide when to reduce learning rate
    min_lr=1e-5                                                         # Minimum learning rate
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=patience,
    restore_best_weights=True
)

callbacks = [early_stopping, lr_scheduler]


# Iterate the configurations

for config in configs:

  config_name = config['config_name']
  resize = config['resize']
  augmentation_enabled = config['augmentation_enabled']
  batch_size = config['batch_size']
  denses = config['denses']
  units = config['units']
  dropout_rate = config['dropout_rate']
  opt_transf_learn = config['opt_transf_learn']
  opt_fine_tuning = config['opt_fine_tuning']
  num_layer_to_freeze = config['num_layer_to_freeze']

  print(f"\n\nConfiguration in use: {config_name}\n\n")

  # Instantiate pretrained model
  pretrained_model = ConvNeXtBase(
    include_top=False,
    weights="imagenet",
  )

  for layer in pretrained_model.layers:
    layer.trainable = False

  # Build the model
  model = build_model(
    pretrained_model,
    resize,
    augmentation_enabled,
    denses,
    units,
    dropout_rate
  )

  # Start the transfer learning phase

  model.compile(
    optimizer=opt_transf_learn,
    loss=keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
  )

  # Start the training in the transfer learning phase and save the history
  tl_history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = batch_size,
    epochs = epochs_transf_learn,
    validation_data=(x_val, y_val),
    callbacks = callbacks
  ).history


  # Start the fine tuning phase

  # Set all ConvNextBase layers to trainable
  model.get_layer('convnext_base').trainable = True
  # Now let's freeze first -num_layer_to_freeze- layers
  for i, layer in enumerate(model.get_layer('convnext_base').layers[:num_layer_to_freeze]):
    layer.trainable=False

  model.compile(
    optimizer=opt_fine_tuning,
    loss=keras.losses.BinaryCrossentropy(),
    metrics=["accuracy"]
  )

  # Start the training in the fine tuning phase and save the history
  ft_history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = batch_size,
    epochs=epochs_fine_tuning,
    validation_data=(x_val, y_val),
    callbacks = callbacks
  ).history


  # Show the plots

  # Plot the transfer learning and the fine-tuned training histories
  plt.figure(figsize=(15,5))
  plt.plot(tl_history['loss'], label='Transfer Learning Training Loss', alpha=.3, color='#4D61E2', linestyle='--')
  plt.plot(tl_history['val_loss'], label='Transfer Learning Validation Loss', alpha=.8, color='#4D61E2')
  plt.plot(ft_history['loss'], label='Fine Tuning Training Loss' ,alpha=.3, color='#408537', linestyle='--')
  plt.plot(ft_history['val_loss'], label='Fine Tuning Validation Loss', alpha=.8, color='#408537')
  plt.legend(loc='upper left')
  plt.title('Binary Crossentropy')
  plt.grid(alpha=.3)

  plt.figure(figsize=(15,5))
  plt.plot(tl_history['accuracy'], label='Transfer Learning Training Accuracy', alpha=.3, color='#4D61E2', linestyle='--')
  plt.plot(tl_history['val_accuracy'], label='Transfer Learning Validation Accuracy', alpha=.8, color='#4D61E2')
  plt.plot(ft_history['accuracy'], label='Fine Tuning Training Accuracy' ,alpha=.3, color='#408537', linestyle='--')
  plt.plot(ft_history['val_accuracy'], label='Fine Tuning Validation Accuracy', alpha=.8, color='#408537')
  plt.legend(loc='upper left')
  plt.title('Accuracy')
  plt.grid(alpha=.3)

  plt.show()

  # Validation Accuracy, F1, Precision, Recall computation

  val_predictions = model.predict(x_val, verbose=0)
  val_predictions = tf.where(val_predictions < 0.5, 0, 1)
  val_predictions = val_predictions.numpy()

  # Compute scores
  val_accuracy = accuracy_score(y_val, val_predictions)
  val_f1 = f1_score(y_val, val_predictions)
  val_precision = precision_score(y_val, val_predictions)
  val_recall = recall_score(y_val, val_predictions)

  # Display the computed metrics
  print('Val Accuracy:', val_accuracy.round(4))
  print('Val F1:', val_f1.round(4))
  print('Val Precision:', val_precision.round(4))
  print('Val Recall:', val_recall.round(4))

  # Save the model
  model.save(config_name)

  # Explicit deletes to avoid waiting the garbage collector
  del config_name, resize, augmentation_enabled, batch_size, denses, units, dropout_rate, opt_transf_learn, opt_fine_tuning, num_layer_to_freeze
  del pretrained_model, model, tl_history, ft_history
  del val_predictions, val_accuracy, val_f1, val_precision, val_recall
